In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [9]:
# Load facilities data
facilities_df = pd.read_csv('facilities_with_risk_scores.csv')

In [10]:
# Convert facilities to GeoDataFrame
facilities_gdf = gpd.GeoDataFrame(
    facilities_df,
    geometry=gpd.points_from_xy(facilities_df.longitude, facilities_df.latitude),
    crs='EPSG:4326'  # WGS84 Latitude/Longitude
)

In [11]:
# Load coastline shapefile
coastline_gdf = gpd.read_file('/Users/prachiheda/Desktop/Florida_Shoreline_(1_to_2%2C000%2C000_Scale)/Florida_Shoreline_(1_to_2%2C000%2C000_Scale).shp')  # Replace with actual path


In [12]:
# Ensure both are in the same coordinate reference system
facilities_gdf = facilities_gdf.to_crs(coastline_gdf.crs)

In [13]:
# Calculate distance to coastline
facilities_gdf['distance_to_coast_km'] = facilities_gdf.geometry.apply(
    lambda x: coastline_gdf.distance(x).min() / 1000  # Convert meters to kilometers
)

In [20]:
# Load hurricane data into a DataFrame
hurricane_data = pd.read_csv('HURDAT2_hurricane_data_with_events.csv')


In [21]:
# Drop rows with missing latitude or longitude
hurricane_data = hurricane_data.dropna(subset=['latitude', 'longitude'])


In [22]:
# Convert latitude and longitude to numeric, coercing errors to NaN
hurricane_data['latitude'] = pd.to_numeric(hurricane_data['latitude'], errors='coerce')
hurricane_data['longitude'] = pd.to_numeric(hurricane_data['longitude'], errors='coerce')


In [23]:
# Convert the DataFrame into a GeoDataFrame
hurricane_gdf = gpd.GeoDataFrame(
    hurricane_data,
    geometry=gpd.points_from_xy(hurricane_data['longitude'], hurricane_data['latitude']),
    crs='EPSG:4326'  # WGS84 Latitude/Longitude
)


In [24]:
# View the first few rows
print(hurricane_gdf.head())

# Check the coordinate reference system (CRS)
print("CRS:", hurricane_gdf.crs)


   storm_id storm_name      date  time record_id event_label status  latitude  \
0  AL011851    UNNAMED  18510625     0       NaN         NaN     HU      28.0   
1  AL011851    UNNAMED  18510625   600       NaN         NaN     HU      28.0   
2  AL011851    UNNAMED  18510625  1200       NaN         NaN     HU      28.0   
3  AL011851    UNNAMED  18510625  1800       NaN         NaN     HU      28.1   
4  AL011851    UNNAMED  18510625  2100         L    Landfall     HU      28.2   

   longitude  max_wind  min_pressure            geometry  
0      -94.8        80          -999    POINT (-94.8 28)  
1      -95.4        80          -999    POINT (-95.4 28)  
2      -96.0        80          -999      POINT (-96 28)  
3      -96.5        80          -999  POINT (-96.5 28.1)  
4      -96.8        80          -999  POINT (-96.8 28.2)  
CRS: EPSG:4326


In [25]:
# Reproject hurricane_gdf to a projected CRS
hurricane_gdf_projected = hurricane_gdf.to_crs(epsg=3086)

In [26]:
# Define buffer distance in meters
buffer_distance = 50000  # 50 km

# Perform the buffering operation
hurricane_gdf_projected['geometry'] = hurricane_gdf_projected.geometry.buffer(buffer_distance)


In [27]:
# Spatial join to count number of hurricanes affecting each facility
facilities_gdf = facilities_gdf.to_crs(hurricane_gdf.crs)
facilities_gdf['hurricane_exposure'] = facilities_gdf.geometry.apply(
    lambda x: hurricane_gdf[hurricane_gdf.geometry.contains(x)]['storm_name'].nunique()
)

In [28]:
# Load the county GeoJSON data
counties_gdf = gpd.read_file('USA_Counties_(Generalized).geojson')


In [29]:
# View the first few rows
print(counties_gdf.head())

# List all column names
print(counties_gdf.columns)

   FID  OBJECTID            NAME    STATE_NAME STATE_FIPS CNTY_FIPS   FIPS  \
0    1         1  Aleutians East        Alaska         02       013  02013   
1    2      1201          Traill  North Dakota         38       097  38097   
2    3      1202           Walsh  North Dakota         38       099  38099   
3    4         2  Aleutians West        Alaska         02       016  02016   
4    5      1203            Ward  North Dakota         38       101  38101   

   POPULATION  POP_SQMI  POP2010  ...  AVE_SIZE12  CROP_ACR12  AVE_SALE12  \
0        3129       0.4     3141  ...         -99         -99         -99   
1        8154       9.5     8121  ...        1170      526183      659887   
2       11037       8.5    11119  ...         834      714525      440783   
3        5609       1.6     5561  ...         -99         -99         -99   
4       75147      36.6    61675  ...        1117      829363      285596   

      SQMI  Shape_Leng  Shape_Area   Shape__Area  Shape__Length  \
0

In [30]:
# Load SVI data
svi_df = pd.read_csv('SVI_2022_US_county.csv', dtype={'FIPS': str})


In [33]:
# Ensure FIPS codes are strings with leading zeros
svi_df['FIPS'] = svi_df['FIPS'].str.zfill(5)

# Assuming the GeoDataFrame has a column named 'GEO_ID' or 'GEOID' for FIPS codes
counties_gdf['FIPS'] = counties_gdf['GlobalID'].str[-5:]


In [34]:
# Merge SVI data with counties GeoDataFrame
merged_gdf = counties_gdf.merge(svi_df[['FIPS', 'RPL_THEMES']], on='FIPS', how='left')


In [35]:
# Filter for Florida counties
florida_gdf = merged_gdf[merged_gdf['STATE_NAME'] == 'Florida'].copy()


In [36]:
# Verify the columns
print(florida_gdf.columns)

# Check for missing SVI data
missing_svi = florida_gdf['RPL_THEMES'].isnull().sum()
print(f"Number of Florida counties without SVI data: {missing_svi}")


Index(['FID', 'OBJECTID', 'NAME', 'STATE_NAME', 'STATE_FIPS', 'CNTY_FIPS',
       'FIPS', 'POPULATION', 'POP_SQMI', 'POP2010', 'POP10_SQMI', 'WHITE',
       'BLACK', 'AMERI_ES', 'ASIAN', 'HAWN_PI', 'HISPANIC', 'OTHER',
       'MULT_RACE', 'MALES', 'FEMALES', 'AGE_UNDER5', 'AGE_5_9', 'AGE_10_14',
       'AGE_15_19', 'AGE_20_24', 'AGE_25_34', 'AGE_35_44', 'AGE_45_54',
       'AGE_55_64', 'AGE_65_74', 'AGE_75_84', 'AGE_85_UP', 'MED_AGE',
       'MED_AGE_M', 'MED_AGE_F', 'HOUSEHOLDS', 'AVE_HH_SZ', 'HSEHLD_1_M',
       'HSEHLD_1_F', 'MARHH_CHD', 'MARHH_NO_C', 'MHH_CHILD', 'FHH_CHILD',
       'FAMILIES', 'AVE_FAM_SZ', 'HSE_UNITS', 'VACANT', 'OWNER_OCC',
       'RENTER_OCC', 'NO_FARMS12', 'AVE_SIZE12', 'CROP_ACR12', 'AVE_SALE12',
       'SQMI', 'Shape_Leng', 'Shape_Area', 'Shape__Area', 'Shape__Length',
       'GlobalID', 'geometry', 'RPL_THEMES'],
      dtype='object')
Number of Florida counties without SVI data: 67


In [39]:
# Convert facilities to GeoDataFrame
facilities_gdf = gpd.GeoDataFrame(
    facilities_df,
    geometry=gpd.points_from_xy(facilities_df['longitude'], facilities_df['latitude']),
    crs='EPSG:4326'
)

In [40]:
# Ensure both GeoDataFrames are in the same CRS
florida_gdf = florida_gdf.to_crs('EPSG:4326')

In [42]:
# Perform spatial join to get SVI scores for facilities
facilities_gdf = gpd.sjoin(
    facilities_gdf,
    florida_gdf[['geometry', 'RPL_THEMES']],
    how='left'
)

In [43]:
facilities_gdf.columns

Index(['name', 'type', 'latitude', 'longitude', 'min_distance', 'risk_score',
       'geometry', 'index_right', 'RPL_THEMES'],
      dtype='object')

In [44]:
# Reproject facilities and coastline to a projected CRS suitable for distance calculation
facilities_projected = facilities_gdf.to_crs(epsg=3086)  # NAD83 / Florida GDL Albers
coastline_projected = coastline_gdf.to_crs(epsg=3086)

In [45]:
# Calculate distance to coastline
facilities_projected['distance_to_coast_m'] = facilities_projected.geometry.apply(
    lambda x: coastline_projected.distance(x).min()
)

In [46]:
# Convert distance to kilometers
facilities_projected['distance_to_coast_km'] = facilities_projected['distance_to_coast_m'] / 1000

In [47]:
# Reproject hurricane data to projected CRS
hurricane_gdf_projected = hurricane_gdf.to_crs(epsg=3086)

In [48]:
# Buffer hurricane paths
buffer_distance = 50000  # 50 km in meters
hurricane_gdf_projected['geometry'] = hurricane_gdf_projected.geometry.buffer(buffer_distance)

In [53]:
print("Facilities columns:", facilities_projected.columns)
print("Hurricane columns:", hurricane_gdf_projected.columns)

Facilities columns: Index(['name', 'type', 'latitude', 'longitude', 'min_distance', 'risk_score',
       'geometry', 'index_right', 'RPL_THEMES', 'distance_to_coast_m',
       'distance_to_coast_km'],
      dtype='object')
Hurricane columns: Index(['storm_id', 'storm_name', 'date', 'time', 'record_id', 'event_label',
       'status', 'latitude', 'longitude', 'max_wind', 'min_pressure',
       'geometry'],
      dtype='object')


In [69]:
# Remove 'index_right' and 'index_left' columns from facilities_projected
facilities_projected = facilities_projected.drop(columns=['index_right', 'index_left'], errors='ignore')

# Remove 'index_right' and 'index_left' columns from hurricane_gdf_projected
hurricane_gdf_projected = hurricane_gdf_projected.drop(columns=['index_right', 'index_left'], errors='ignore')


In [55]:
facilities_projected = facilities_projected.reset_index(drop=True)
hurricane_gdf_projected = hurricane_gdf_projected.reset_index(drop=True)


In [64]:
hurricane_gdf_projected.head()

,storm_id,storm_name,date,time,record_id,event_label,status,latitude,longitude,max_wind,min_pressure,geometry
0,AL011851,UNNAMED,18510625,0,NaN,NaN,HU,28.0,-94.8,80,-999,"POLYGON ((-608696.456 490186.163, -608937.22 4..."
1,AL011851,UNNAMED,18510625,600,NaN,NaN,HU,28.0,-95.4,80,-999,"POLYGON ((-667349.703 495480.069, -667590.467 ..."
2,AL011851,UNNAMED,18510625,1200,NaN,NaN,HU,28.0,-96.0,80,-999,"POLYGON ((-725976.498 501059.296, -726217.261 ..."
3,AL011851,UNNAMED,18510625,1800,NaN,NaN,HU,28.1,-96.5,80,-999,"POLYGON ((-773687.14 516975.086, -773927.904 5..."
4,AL011851,UNNAMED,18510625,2100,L,Landfall,HU,28.2,-96.8,80,-999,"POLYGON ((-801800.629 531033.553, -802041.393 ..."


In [70]:
# Spatial join to calculate exposure
facilities_projected = gpd.sjoin(
    facilities_projected,
    hurricane_gdf_projected[['geometry', 'storm_name']],
    how='left',
    predicate='intersects'
)


In [59]:
# Since the index represents facilities, group by the index
facilities_projected['hurricane_exposure'] = facilities_projected.groupby(level=0)['storm_name'].transform('nunique').fillna(0)


In [71]:
facilities_projected.head(10)

,name,type,latitude,longitude,min_distance,risk_score,geometry,RPL_THEMES,distance_to_coast_m,distance_to_coast_km,storm_name_left,hurricane_exposure,index_right,storm_name_right
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,7106,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,631,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,804,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,29043,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,15719,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,29042,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,632,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,25857,FLORENCE
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,2453,UNNAMED
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613,POINT (551369.498 439026.651),NaN,0.0,0.0,UNNAMED,8,16013,UNNAMED


In [60]:
# Prepare the final DataFrame
model_df = facilities_projected.dropna(subset=['distance_to_coast_km', 'hurricane_exposure', 'RPL_THEMES', 'risk_score'])

In [61]:
model_df.head()

,name,type,latitude,longitude,min_distance,risk_score,geometry,RPL_THEMES,distance_to_coast_m,distance_to_coast_km,index_right,storm_name,hurricane_exposure


In [ ]:
# Define features and target
features = ['distance_to_coast_km', 'hurricane_exposure', 'RPL_THEMES']
X = model_df[features]
y = model_df['risk_score']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
